In [35]:
#pip install pandas-datareader
import pandas as pd
import pandas_datareader.data as web
import pandas_datareader as pdr
import seaborn as sns
import datetime
import numpy as np
import matplotlib.pyplot as plt

In [36]:
#setting section
money = 100000
start = datetime.datetime(2016, 1, 1)
finish = datetime.datetime(2020, 12, 31)
stop_loss = 7
short_window = 30
long_window = 90

prod = 'sber' #'IBM'
source = 'moex' #'yahoo'

In [37]:
hist_inf = web.DataReader(prod, source, start=start, end=finish)

col_labels = {
    'OPEN':'Open',
    'open':'Open',
    'HIGH':'High',
    'high':'High',
    'LOW':'Low',
    'low':'Low',
    'CLOSE':'Close',
    'close':'Close',
}

hist_inf_r = hist_inf.rename(columns=col_labels)
hist_inf_r.index.name = 'Date'
hist_com = hist_inf_r[['Open', 'Low', 'High', 'Close']]

hist_com.head()

,Open,Low,High,Close
Date,,,,
2016-01-04,101.00,97.94,101.78,98.01
2016-01-05,97.88,97.16,99.24,98.89
2016-01-06,98.70,96.83,99.56,96.85
2016-01-11,93.61,91.01,94.89,91.03
2016-01-12,89.09,88.52,92.88,91.71


In [38]:
ma_short = hist_com['Close'].rolling(short_window, min_periods=1).mean()
ma_long = hist_com['Close'].rolling(long_window, min_periods=1).mean()
result = pd.concat([hist_com['Close'], ma_short, ma_long], axis=1)
result.columns=['share_price','ma_'+ str(short_window) ,'ma_'+ str(long_window)]
#result.plot()

In [39]:
df_result = pd.merge(hist_com, result, on = 'Date')
df_result = df_result.drop(columns=['share_price'])
df_result = df_result.reset_index(level=0)


df_result['signal_test'] = 0.0
df_result['signal_test'] = np.where(df_result['ma_'+ str(short_window)] > df_result['ma_'+ str(long_window)], 1.0, 0.0)

df_result['position'] = df_result['signal_test'].diff()
df_result['position_t'] = df_result['signal_test'].diff()

df_result.loc[df_result['position_t'] == 1.0, 'position_t'] = 'buy'

df_result['signal_t'] = df_result['position_t']
df_result['position_t'] = df_result.position_t.shift(1)

df_result.loc[df_result['signal_t'] == 'buy', 'signal_t'] = 'sig_buy'

df_result['stopper'] = np.where(df_result['position_t'] == 'buy', df_result['Open'], 0.0)
df_result.loc[df_result['stopper'] == 0, 'stopper'] = np.nan
df_result['stopper'] = df_result['stopper'].fillna(method="ffill")

df_result['signal_stop'] = np.where((df_result['Open'] <= (((100-stop_loss)/100) * df_result['stopper'])) | 
                                    (df_result['Low'] <= (((100-stop_loss)/100) * df_result['stopper'])) |
                                    (df_result['Close'] <= (((100-stop_loss)/100) * df_result['stopper'])), 1.0, 0.0)

df_result['stopper_i'] = np.where(df_result['position_t'] == 'buy', df_result.index, 0.0)
df_result.loc[df_result['stopper_i'] == 0, 'stopper_i'] = np.nan
df_result['stopper_i'] = df_result['stopper_i'].fillna(method="ffill")

df_result['stopper_loss_i'] = np.where((df_result['position_t'] == 'buy') | (df_result['signal_stop'] == 1.0),
                                        df_result.index, 0.0)
df_result.loc[df_result['stopper_loss_i'] == 0, 'stopper_loss_i'] = np.nan
df_result['stopper_loss_i'] = df_result['stopper_loss_i'].fillna(method="ffill")

df_result['paper_in_1'] = df_result.apply(lambda row: 1.0 if row['stopper_loss_i'] == row['stopper_i'] else 0.0, axis=1)
df_result['paper_in_2'] = df_result.paper_in_1.shift(1)

df_result['signal_test_2'] = df_result.signal_test.shift(1)
df_result['paper_in'] = np.where(((df_result['paper_in_1'] == 1.0) | (df_result['paper_in_2'] == 1.0)) & 
                                 ((df_result['signal_test_2'] == 1.0)), 1.0, 0.0)

df_result['signal_stop'] = np.where((df_result['signal_stop'] == 1.0) & (df_result['paper_in'] == 1.0), 1.0, 0.0)

df_result.loc[df_result['position'] == 1.0, 'position'] = 'buy'
df_result.loc[(df_result['position'] == -1.0 ) & (df_result['paper_in'] == 1.0), 'position'] = 'sale'
df_result.loc[(df_result['position'] == -1.0 ) & (df_result['paper_in'] == 0.0), 'position'] = 0.0

df_result['signal_1'] = df_result['position']
df_result['position'] = df_result.position.shift(1)

df_result.loc[df_result['signal_1'] == 'buy', 'signal_1'] = 'sig_buy'
df_result.loc[df_result['signal_1'] == 'sale', 'signal_1'] = 'sig_sale'
df_result.loc[df_result['signal_stop'] == 1.0, 'signal_stop'] = 'stop_loss'

df_result['signal_2'] = df_result.apply(lambda row: row['signal_1'] if row['position'] == 0 else row['position'], axis=1) 
df_result['signal'] = df_result.apply(lambda row: row['signal_stop'] if row['signal_2'] == 0 else row['signal_2'], axis=1) 

df_result_for_plot = df_result

df_result = df_result.drop(columns=['position', 'signal_1', 'signal_stop', 'signal_2', 'paper_in_1', 'paper_in_2', 'stopper_i',
                                    'stopper_loss_i', 'signal_test_2', 'position_t', 'signal_t'])
df_result = df_result[df_result.signal != 0]
df_result = df_result[df_result['signal'].notna()]
df_result = df_result.reset_index()
df_result = df_result.drop(columns=['index'])
    
#df_result

In [40]:
df_result['share_value'] = np.where((df_result['signal'] == 'stop_loss') &
                                   ((df_result['Open'] <= (((100-stop_loss)/100) * df_result['stopper']))), 
                                    round(df_result['Open'], 2), round(df_result['Low'], 2))
df_result['share_value'] = np.where((df_result['signal'] == 'sig_buy')|(df_result['signal'] == 'sig_sale'), 
                                    round(df_result['Close'], 2), df_result['share_value'])
df_result['share_value'] = np.where((df_result['signal'] == 'buy')|(df_result['signal'] == 'sale'), 
                                     round(df_result['Open'], 2), df_result['share_value'])

num_shares = 0
share_price = 0
rest = 0
cash = money

for i in range(len(df_result)):
    
    if df_result.loc[i, 'signal'] == 'sig_buy':
        df_result.loc[i, 'num_shares'] = 0
        df_result.loc[i, 'rest'] = rest
        df_result.loc[i, 'cash'] = cash  
        
    if df_result.loc[i, 'signal'] == 'buy':
        num_shares = cash/df_result.loc[i, 'share_value']
        num_shares = num_shares.astype(int)
        df_result.loc[i, 'num_shares'] = num_shares
        share_price = num_shares * df_result.loc[i, 'share_value']
        df_result.loc[i, 'share_price'] = share_price
        rest = cash - share_price
        df_result.loc[i, 'rest'] = rest
        cash = rest
        df_result.loc[i, 'cash'] = cash
        rest = 0
        
    if df_result.loc[i, 'signal'] == 'sig_sale':
        df_result.loc[i, 'num_shares'] = num_shares
        df_result.loc[i, 'rest'] = rest
        df_result.loc[i, 'cash'] = cash  
           
        
    if (df_result.loc[i, 'signal'] == 'sale')|(df_result.loc[i, 'signal'] == 'stop_loss'):
        df_result.loc[i, 'num_shares'] = num_shares
        share_price = num_shares * df_result.loc[i, 'share_value']
        df_result.loc[i, 'share_price'] = share_price
        df_result.loc[i, 'rest'] = rest
        cash = cash + df_result.loc[i, 'share_price']
        df_result.loc[i, 'cash'] = cash 
      
        
df_final_result = df_result[['Date', 'signal', 'num_shares', 'share_value', 'share_price', 'rest', 'cash']]
df_result_for_cash = df_result

mask = (df_result_for_cash['signal'] == 'sale')|(df_result_for_cash['signal'] == 'stop_loss')
df_result_for_cash = df_result_for_cash[mask]
final_cash = df_result_for_cash.loc[df_result_for_cash.index[-1], 'cash']

print('Стартовый капитал: ', money)
print('В конце периода на руках: ', round(final_cash, 2))
print('Заработано: ', round(final_cash-money, 2))
print('Таким образом, прирост составил: ', round((final_cash-money)*100/money, 2), '%')
df_final_result

Стартовый капитал:  100000
В конце периода на руках:  159989.1
Заработано:  59989.1
Таким образом, прирост составил:  59.99 %


,Date,signal,num_shares,share_value,share_price,rest,cash
0,2016-02-24,sig_buy,0.0,102.10,NaN,0.00,100000.00
1,2016-02-25,buy,974.0,102.65,99981.10,18.90,18.90
2,2017-03-15,sig_sale,974.0,156.20,NaN,0.00,18.90
3,2017-03-16,sale,974.0,157.82,153716.68,0.00,153735.58
4,2017-06-01,sig_buy,0.0,153.00,NaN,0.00,153735.58
5,2017-06-02,buy,1004.0,153.00,153612.00,123.58,123.58
6,2017-06-13,sig_sale,1004.0,146.00,NaN,0.00,123.58
7,2017-06-14,sale,1004.0,145.65,146232.60,0.00,146356.18
8,2017-08-07,sig_buy,0.0,172.79,NaN,0.00,146356.18
9,2017-08-08,buy,845.0,173.00,146185.00,171.18,171.18


In [41]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.9, 0.1])

fig.add_trace(go.Ohlc(x=df_result_for_plot['Date'], open=df_result_for_plot['Open'], 
                                     high=df_result_for_plot['High'], low=df_result_for_plot['Low'],
                                     close=df_result_for_plot['Close'], name='share price'), row=1, col=1)

fig.add_trace(go.Scatter(x=df_result_for_plot['Date'], y=df_result_for_plot['ma_'+ str(short_window)], 
                                 name='ma_'+ str(short_window), line=dict(color='orange', width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df_result_for_plot['Date'], y=df_result_for_plot['ma_'+ str(long_window)], 
                                 name='ma_'+ str(long_window), line=dict(color='green', width=1)), row=1, col=1)

df_result_for_plot_position = df_result_for_plot
df_result_for_plot_position['signal_test'] = df_result_for_plot_position.signal_test.shift(1)

fig.add_trace(go.Scatter(
        x=df_result.loc[df_result['signal'] == 'sig_buy', 'Date'],
        y=df_result.loc[df_result['signal'] == 'sig_buy', 'Open'],
        mode="markers+text",
        marker=dict(symbol='triangle-up', color='green', size = 12),
        name='sig_buy'), row=1, col=1)

fig.add_trace(go.Scatter(
        x=df_result.loc[df_result['signal'] == 'sig_sale', 'Date'],
        y=df_result.loc[df_result['signal'] == 'sig_sale', 'Open'],
        mode="markers+text",
        marker=dict(symbol='triangle-down', color='red', size = 12),
        name='sig_sale'), row=1, col=1)

fig.add_trace(go.Scatter(
        x=df_result.loc[df_result['signal'] == 'stop_loss', 'Date'],
        y=df_result.loc[df_result['signal'] == 'stop_loss', 'Open'],
        mode="markers+text",
        marker=dict(symbol='triangle-down', color='black', size = 12),
        name='stop_loss'), row=1, col=1)

fig.add_trace(go.Scatter(x=df_result_for_plot_position['Date'],
                         y=df_result_for_plot_position['paper_in'],
                         line=dict(color='black', width=1), name='position'), row=2, col=1)

fig.update(layout_xaxis_rangeslider_visible=False)


fig.show()